# Switching Assistant
- Intent classification and Service Routing
- The LLM analyzes this combination and classifies the input into one or combination of three main intents: AppAssistant, PainAssistant, or PersonalAssistant.
- Creating through bedrock-agent for more confirugation and control

In [1]:
import boto3
import sys
import json
import pprint
from botocore.client import Config
from botocore.exceptions import NoCredentialsError, PartialCredentialsError, ClientError
import os
import random
from retrying import retry
import time
from utility import *
print('Running boto3 version:', boto3.__version__)

Running boto3 version: 1.35.34


In [ ]:
# Bedrock Agent runtime
pp = pprint.PrettyPrinter(indent=2)
session = boto3.session.Session()
region = 'ca-central-1'
bedrock_config = Config(connect_timeout=120, read_timeout=120, retries={'max_attempts': 0})
bedrock_client = boto3.client('bedrock-runtime', region_name = region)
bedrock_agent_client = boto3.client("bedrock-agent-runtime", config=bedrock_config, region_name = region)

# KB ID
kb_id = 'V2W0LT3GZP'

In [ ]:
def retrieve(query, kbId, numberOfResults=5):
    return bedrock_agent_client.retrieve(
        retrievalQuery= {
            'text': query
        },
        knowledgeBaseId=kbId,
        retrievalConfiguration= {
            'vectorSearchConfiguration': {
                'numberOfResults': numberOfResults,
                'overrideSearchType': "HYBRID", # optional
            }
        }
    )

def get_contexts(retrievalResults):
    contexts = []
    for retrievedResult in retrievalResults: 
        contexts.append(retrievedResult['content']['text'])
    return contexts

In [ ]:
# Assigning the default knowledge base prompt

path = '../machine-learning/4. Knowledge Base Template/'
filename = 'Switching_Assistant_V1_Oct17.md'

with open(path+filename, 'r') as f:
    prompt_template = f.read()


Evaluating role switching on Potential Questions 2 (which is supposed to be only for app assistant)

In [ ]:
# Loading the questions that we want to ask

path = '../machine-learning/6. Test Cases/App Assistant/'
filename = 'Potential-Questions2.md'

with open(path+filename, 'r') as f:
    questions = f.read()

queries = questions.split('\n')


# Going through the queries to send to LLM

for query in queries:
    print ('---------------------------------------------------- \n')

    # Acquire the context
    response = retrieve(query, kb_id)
    retrievalResults = response['retrievalResults']
    contexts = get_contexts(retrievalResults)

    # Create the prompt
    prompt = f"""
    Human: Use the following <context> to determine if the intent of <query> is classified as AI App assistant or as the 
    Pain assistant as decribed in <prompt_template>
    
    <prompt_template>
    {prompt_template}
    </prompt_template>
    
    <context>
    {contexts}
    </context>
    
    <query>
    {query}
    </query>

    Only respond with the title name of the assistant chosen.
    Do not respond if the prompt is not related to <contexts>.
    
    Assistant:"""

    # Prepare the information to be invoked into the model
    messages=[{ "role":'user', "content":[{'type':'text','text': prompt.format(prompt_template, contexts, query)}]}]
    sonnet_payload = json.dumps({
        "anthropic_version": "bedrock-2023-05-31",
        "max_tokens": 512,
        "messages": messages,
        "temperature": 1,
        "top_p": 1
            }  )
    modelId = 'anthropic.claude-3-sonnet-20240229-v1:0' 
    accept = 'application/json'
    contentType = 'application/json'
    response = bedrock_client.invoke_model(body=sonnet_payload, 
                                           modelId=modelId, 
                                           accept=accept, 
                                           contentType=contentType)

    # Call and print the response
    response_body = json.loads(response.get('body').read())
    response_text = response_body.get('content')[0]['text']
    # pp.pprint (query)
    # pp.pprint(response_text)
    print (query)
    print (response_text)
    

--- 
# Test the role switching from app assistant to pain pscyhology

In [ ]:
# Loading the questions that we want to ask

path = '../machine-learning/6. Test Cases/'
filename = 'Role-Switching2.md'

with open(path+filename, 'r') as f:
    questions = f.read()

queries = questions.split('\n')


# Going through the queries to send to LLM

for query in queries:
    print ('---------------------------------------------------- \n')

    # Acquire the context
    response = retrieve(query, kb_id)
    retrievalResults = response['retrievalResults']
    contexts = get_contexts(retrievalResults)

    # Create the prompt
    prompt = f"""
    Human: Use the following <context> to determine if the intent of <query> is classified as AI App assistant or as the 
    Pain assistant as decribed in <prompt_template>. 
    
    <prompt_template>
    {prompt_template}
    </prompt_template>
    
    <context>
    {contexts}
    </context>
    
    <query>
    {query}
    </query>

    Only respond with the title name of the assistant(s) chosen. 
    Do not respond if the prompt is not related to <contexts>.
    
    Assistant:"""

    # Prepare the information to be invoked into the model
    messages=[{ "role":'user', "content":[{'type':'text','text': prompt.format(prompt_template, contexts, query)}]}]
    sonnet_payload = json.dumps({
        "anthropic_version": "bedrock-2023-05-31",
        "max_tokens": 512,
        "messages": messages,
        "temperature": 1,
        "top_p": 1
            }  )
    modelId = 'anthropic.claude-3-sonnet-20240229-v1:0' 
    accept = 'application/json'
    contentType = 'application/json'
    response = bedrock_client.invoke_model(body=sonnet_payload, 
                                           modelId=modelId, 
                                           accept=accept, 
                                           contentType=contentType)

    # Call and print the response
    response_body = json.loads(response.get('body').read())
    response_text = response_body.get('content')[0]['text']
    # pp.pprint (query)
    # pp.pprint(response_text)
    print (query)
    print (response_text)
    

--- 
# Role switching extension

In [27]:
# Bedrock Agent runtime
pp = pprint.PrettyPrinter(indent=2)
session = boto3.session.Session()
region = 'ca-central-1'
bedrock_config = Config(connect_timeout=120, read_timeout=120, retries={'max_attempts': 0})
bedrock_client = boto3.client('bedrock-runtime', region_name = region)
bedrock_agent_client = boto3.client("bedrock-agent-runtime", config=bedrock_config, region_name = region)

In [28]:
def retrieve(query, kbId, numberOfResults=5):
    return bedrock_agent_client.retrieve(
        retrievalQuery= {
            'text': query
        },
        knowledgeBaseId=kbId,
        retrievalConfiguration= {
            'vectorSearchConfiguration': {
                'numberOfResults': numberOfResults,
                'overrideSearchType': "HYBRID", # optional
            }
        }
    )

def get_contexts(retrievalResults):
    contexts = []
    for retrievedResult in retrievalResults: 
        contexts.append(retrievedResult['content']['text'])
    return contexts

In [34]:
def prompt_choice (prompt_template, query, contexts, choice):

        if (choice == 'Intent'):
            prompt = f"""
            Human: Use the following <context> to determine if the intent of <query> is classified as AI App assistant or as the 
            Pain assistant as decribed in <prompt_template>. 

            If the intent is classified as AI App assistant then respond with 'AI Assistant for the App', 
            if the intent is classified as the Pain assistant then respond with 'Pain Psychologist',
            if the intent is classifed as both the AI App assistant and Pain assistant then respond with 'Both'
            
            <prompt_template>
            {prompt_template}
            </prompt_template>
            
            <context>
            {contexts}
            </context>
            
            <query>
            {query}
            </query>
        
            Only respond with the title name of the assistant(s) chosen. 
            Do not respond if the prompt is not related to <contexts>.
            
            Assistant:"""
            
        elif (choice == 'Pain'):
            prompt = f"""
            Human: Use the following <context> to answer the <query> according to your role in <prompt_template>. 
            <prompt_template>
            {prompt_template}
            </prompt_template>
            
            <context>
            {contexts}
            </context>
            
            <query>
            {query}
            </query>
            
            Do not respond if the prompt is not related to pain Psychology.
            Do not respond if the prompt is not related to <contexts>.
            Do not respond if the prompt is related to usage of the app.
            
            Assistant:"""


        elif (choice == 'App'):
            prompt = f"""
            Human: Use the following <context> to answer the <query> according to your role in <prompt_template>. 
            <prompt_template>
            {prompt_template}
            </prompt_template>
            
            <context>
            {contexts}
            </context>
            
            <query>
            {query}
            </query>
            
            Do not respond if the prompt is not related to the app.
            
            Assistant:"""
            

        return (prompt)


def prompt_template_choice(choice):
    # Assigning the default knowledge base prompt
    path = '../machine-learning/4. Knowledge Base Template/'

    if (choice == 'Intent'):
        filename = 'Switching_Assistant_V1_Oct17.md'

    elif (choice == 'Pain'):
        filename = 'Pain_Assistant_V1_Oct15.md'
            
    elif (choice == 'App'):
        filename = 'App_Assistant_V1_Oct10.md'
        
    with open(path+filename, 'r') as f:
        prompt_template = f.read()
            
    return (prompt_template)



In [30]:
def data_ingestion (prompt, prompt_template, contexts, query, temp = 1):

    # Prepare the information to be invoked into the model
    messages=[{ "role":'user', "content":[{'type':'text','text': prompt.format(prompt_template, contexts, query)}]}]
    sonnet_payload = json.dumps({
    "anthropic_version": "bedrock-2023-05-31",
    "max_tokens": 512,
    "messages": messages,
    "temperature": temp,
    "top_p": 1
        }  )
    modelId = 'anthropic.claude-3-sonnet-20240229-v1:0' 
    accept = 'application/json'
    contentType = 'application/json'
    response = bedrock_client.invoke_model(body=sonnet_payload, 
                                       modelId=modelId, 
                                       accept=accept, 
                                       contentType=contentType)
    
    return (response)

In [31]:
# Knowledge base ID

kb_aws_bedorck_s3 = '76UIT87ACB'
kb_app_assistant = 'V2W0LT3GZP'
kb_pain_assistant = 'KESDTCXEJE'

In [32]:
# Loading the questions that we want to ask

path = '../machine-learning/6. Test Cases/'
filename = 'Role-Switching2.md'

with open(path+filename, 'r') as f:
    questions = f.read()

queries = questions.split('\n')

In [35]:

for query in queries:
    
    # Main body of the analysis
    
    print ('xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx ')
    print ('Begin LLM for Intent Classification')
    print ('---------------------------------------------------- \n')
    
    # Acquire the context
    response = retrieve(query, kb_aws_bedorck_s3)
    retrievalResults = response['retrievalResults']
    contexts = get_contexts(retrievalResults)
    
    # Choose the prompt template
    prompt_template = prompt_template_choice('Intent')
    
    # Format the Prompt with the prompt template
    prompt_role_switching = prompt_choice (prompt_template, query, contexts, 'Intent')
    
    # Ingest data into the LLM (Modifications can be applied here)
    response = data_ingestion (prompt_role_switching, prompt_template, contexts, query, 0)
    
    # Call and print the response
    response_body = json.loads(response.get('body').read())
    response_text = response_body.get('content')[0]['text']
    print (query)
    print (response_text)
    
    print ('\n---------------------------------------------------- ')
    print ('End of intent classification')
    print ('---------------------------------------------------- ')
    
    if (response_text == 'AI Assistant for the App'):
        print ('Begin LLM for AI Assistant')
        print ('---------------------------------------------------- \n')
        
        # Acquire the context
        response = retrieve(query, kb_app_assistant)
        retrievalResults = response['retrievalResults']
        contexts = get_contexts(retrievalResults)
        
        # Choose the prompt template
        prompt_template = prompt_template_choice('App')
        
        # Format the Prompt with the prompt template
        prompt_role_switching = prompt_choice (prompt_template, query, contexts, 'App')
        
        # Ingest data into the LLM (Modifications can be applied here)
        response = data_ingestion (prompt_role_switching, prompt_template, contexts, query)
        
        # Call and print the response
        response_body = json.loads(response.get('body').read())
        response_text = response_body.get('content')[0]['text']
        print (response_text)
        
        print ('\n ---------------------------------------------------- ')
        print ('End of App AI Assistant')
        print ('xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx ')
        
    
    elif (response_text == 'Pain Psychologist'):
        print ('Begin LLM for Pain Assistant')
        print ('---------------------------------------------------- \n')
        
        # Acquire the context
        response = retrieve(query, kb_pain_assistant)
        retrievalResults = response['retrievalResults']
        contexts = get_contexts(retrievalResults)
        
        # Choose the prompt template
        prompt_template = prompt_template_choice('Pain')
        
        # Format the Prompt with the prompt template
        prompt_role_switching = prompt_choice (prompt_template, query, contexts, 'Pain')
        
        # Ingest data into the LLM (Modifications can be applied here)
        response = data_ingestion (prompt_role_switching, prompt_template, contexts, query)
        
        # Call and print the response
        response_body = json.loads(response.get('body').read())
        response_text = response_body.get('content')[0]['text']
        print (response_text)
        
        print ('\n----------------------------------------------------')
        print ('End of Pain AI Assistant')
        print ('xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx ')
        

xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx 
Begin LLM for Intent Classification
---------------------------------------------------- 

How do I update my pain condition in the app? How does stress or anxiety affect my pain levels?
AI Assistant for the App, Pain Psychologist

---------------------------------------------------- 
End of intent classification
---------------------------------------------------- 
xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx 
Begin LLM for Intent Classification
---------------------------------------------------- 

Can I customize the frequency of notifications?
AI Assistant for the App

---------------------------------------------------- 
End of intent classification
---------------------------------------------------- 
Begin LLM for AI Assistant
---------------------------------------------------- 

Yes, you can customize the frequency of notifications in the Manage My Pain app. Here are the steps:

1. Tap on the Settings icon in the ap